In [ ]:
"""
*Uncomment if running on colab* 
Set Runtime -> Change runtime type -> Under Hardware Accelerator select GPU in Google Colab 
"""
# !git clone https://github.com/DmitryUlyanov/deep-image-prior
# !mv deep-image-prior/* ./

# Import libs

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models import *

import torch
import torch.optim
import torch.nn as nn
import torch.nn.init

from torchsummary import summary

from skimage.measure import compare_psnr
from models.downsampler import Downsampler

from utils.sr_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

imsize = -1 
factor = 4 # 8
enforse_div32 = 'CROP' # we usually need the dimensions to be divisible by a power of two (32 in this case)
PLOT = True

# To produce images from the paper we took *_GT.png images from LapSRN viewer for corresponding factor,
# e.g. x4/zebra_GT.png for factor=4, and x8/zebra_GT.png for factor=8 
path_to_image = 'data/sr/zebra_GT.png'

# Load image and baselines

In [ ]:
# Starts here
imgs = load_LR_HR_imgs_sr(path_to_image , imsize, factor, enforse_div32)

imgs['bicubic_np'], imgs['sharp_np'], imgs['nearest_np'] = get_baselines(imgs['LR_pil'], imgs['HR_pil'])

if PLOT:
    plot_image_grid([imgs['HR_np'], imgs['bicubic_np'], imgs['sharp_np'], imgs['nearest_np']], 4,12);
    print ('PSNR bicubic: %.4f   PSNR nearest: %.4f' %  (
                                        compare_psnr(imgs['HR_np'], imgs['bicubic_np']), 
                                        compare_psnr(imgs['HR_np'], imgs['nearest_np'])))

# CNN used in Echigo's paper

In [ ]:
class ESPCN(nn.module):
    only_luminance = True
    input_upscale = False

    def __init__(self, upscale=2):
        super(ESPCN, self).__init__()
        self.upscale = upscale
        self.conv1 = nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(4, upscale ** 2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(upscale ** 2, 40, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1)
        self.upsampler = nn.Upsample(scale_factor=2, mode='bilenear')
        self.flatten = nn.Flatten()
        self.pixel_shuffle = nn.PixelShuffle(self.upscale)
        self.features = nn.Sequential(
            self.conv1,
            self.conv2,
            self.pixel_shuffle,
            self.upsampler,
            self.conv3,
            self.conv4,
            self.flatten
        )

    def _initialize_weights(self):
        weights_with_relu = [
            self.conv1.weight,
            self.conv2.weight,
            self.conv3.weight
        ]

        for w in weights_with_relu:
            nn.init.orthogonal_(w, nn.init.calculate_gain('relu'))

        nn.init.orthogonal_(self.conv4.weight)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = ESPCN().to(device)
summary(net, (3, 32, 32))